In [13]:
import firebase_admin
from firebase_admin import firestore
import os
import pandas as pd
from pathlib import Path



In [4]:
cred = firebase_admin.credentials.Certificate('key.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()
instructions_ref = db.collection('instructions')
verify_ref = db.collection('verification')


In [5]:
list_instruction = list(instructions_ref.get())
list_verification = list(verify_ref.get())
len(list_instruction), len(list_verification)


(3171, 3374)

In [17]:
# Qualified workers
current_dir = os.getcwd()
parent_dir = Path(current_dir).parent.absolute()
path_qualified = os.path.join(parent_dir, "qualified_workers.csv")

qualified_work_list = pd.read_csv(path_qualified, header=None)[0].tolist()

qualified_work_list

['1',
 'AY7WPVKHVNBLG',
 'A1198W1SPF1R4',
 'A3QAL3X23LHJRV',
 'A3GWRDHAURRNK6',
 'A2DDPSXH2X96RF',
 'ALYR5CI2SM2JC',
 'A320QA9HJFUOZO',
 'APRZ7BR8C0ZMQ',
 'A3862RIFFUV141',
 'A1AKL5YH9NLD2V',
 'A2WX434EAQOE29',
 'AYIFHDQSXQJ6B']

In [19]:
# Set qualified instructions as RVS_excellent 
qualified_instructions = [
  e.to_dict()['key'] for e in list_instruction if (
    e.to_dict()['work_id'] in qualified_work_list) ]

for id in qualified_instructions:
  instructions_ref.document(id).update(
    {'review': 'RVS_excellent'})


In [ ]:
# Split instructions to train, dev, test sets.
import json

instruction_dir = "/mnt/hackney/human_data"

def save_split(split_name, split_set):

  split_path = os.path.join(instruction_dir, split_name)
  if os.path.exists(split_path):
    os.remove(split_path)
  with open(split_path, 'a') as outfile:
    for sample in split_set:
      json.dump(sample, outfile, default=lambda o: o.__dict__)
      outfile.write('\n')
      outfile.flush()

  print (f"Size of {split_name} set:{len(split_set)} saved to: {split_path}")


# Save set splits

dev_ids = ['X']
test_ids = ['X']


test_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in test_ids]

save_split("test.json", test_set)

dev_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in dev_ids]

save_split("dev.json", dev_set)

train_set = [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] not in dev_ids+test_ids]

save_split("train.json", train_set)


In [20]:
# check the new workers ID
df_instruction = pd.DataFrame(
    [
        x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and  'date_start' in x.to_dict() and '2022-11-01' <x.to_dict()['date_start'] ])

df_verification = pd.DataFrame(
        [x.to_dict() for x in list_verification] )


df = df_instruction.merge(df_verification, left_on='key', right_on='key_instruction')

work_ids = list(set(df['work_id_x'].to_list()))


for work_id in work_ids:
    if work_id in qualified_work_list:
        continue
    print (f"--------{work_id}--------")
    print (df[df['work_id_x']==work_id][['date_start_x', 'review', 'hit_id_x','dist_m']])


--------A1FS8SBR4SDWYG--------
                  date_start_x         review                        hit_id_x  \
34  2022-11-21 16:16:03.310371  RVS_excellent  373L46LKP779OZDA4FAYCEM5VMHKJE   

    dist_m  
34    71.0  
--------A39MOCMDRIOSJM--------
                   date_start_x         review  \
245  2022-12-02 11:51:32.243747  RVS_excellent   

                           hit_id_x  dist_m  
245  3FBEFUUYRK6RORPH853K9WE449Y6AK    21.0  
--------AZLZA0Q87TJZO--------
                   date_start_x review                        hit_id_x  dist_m
572  2022-11-30 14:47:31.865705    NaN  3K8CQCU3KE2W2PYC8RLTTWXHRHBWNE     4.0


In [6]:
all_data = pd.DataFrame([x.to_dict() for x in list_instruction ])
x = all_data[all_data['review']=='RVS_excellent' ]
x[x['verified_n']==0].shape

NameError: name 'pd' is not defined